In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Text extraction from Image

Text detection is the process of determining the location of text in an image.Text detection can be thought of as a subset of object detection.
I am using Pytesseract library to extract the text.
if you really like it then upvote.









# Import All Libraries

In [ ]:
from PIL import Image
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import re,string,unicodedata
import gensim.parsing.preprocessing as gsp
from gensim import utils
from textblob import TextBlob, Word
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
import pytesseract
import gc


In [ ]:
#read the sentiment dataset
image_data = '../input/detecting-sentiments-dataset/Sample Data Files/'
data_test = '../input/detecting-sentiments-dataset/Dataset/'

# Text Extraction 

In [ ]:
txt = []

#extracting the text 
def TxtExtract(directory):
    
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            filepath = subdir + os.sep + file
            text = pytesseract.image_to_string(Image.open(filepath), timeout=5)
            if not text:
                txt.extend([[file, "blank"]])
            else:   
                txt.extend([[file, text]])
                
    fol_nm = os.path.split(os.path.dirname(subdir))[-1]
    
    print(f"Text Extracted from the images '{fol_nm}' folder & saved to list..")


# Text Extracted from Image

In [ ]:
TxtExtract(image_data)
TxtExtract(data_test)

# Created a DataFrame with text data after extraction

In [ ]:
df = pd.DataFrame(txt,columns=['Image_Name','text_data'])


# Text Preprocessing

In [ ]:
# df['text_data'].apply(str)
# df['text_data']=df['text_data'].fillna("na").values
#Text preprocessing
# import re
# #train_data = text_data.reset_index(drop=True)
# REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
# BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# STOPWORDS = set(stopwords.words('english'))

# def clean_text(text):
#     """
#         text: a string
        
#         return: modified initial string
#     """
#     text = text.lower() # lowercase text
#     text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
#     text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#     text = text.replace('text', ' ')
#     text =" ".join([i for i in text if i.isdigit()])
#     text = re.sub(r'\W+', '', text)
#     text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
#     return text
# df['text_data'] = df['text_data'].apply(clean_text)

# Creating Sentiment using Textblob

In [ ]:
# creating sentiment using Textblob

def data_senti(text):
    TB_sentiment_polarity = TextBlob(text).sentiment.polarity
    
    # decide sentiment as positive, negative and neutral 
    if TB_sentiment_polarity >= 0.00 : 
        return "Positive" 
  
    elif TB_sentiment_polarity <= 0.00 : 
        return "Negative" 
  
    else : 
        return "Neutral"

In [ ]:
df['label'] = df['text_data'].apply(lambda x: data_senti(x))

# Word_Cloud implementation

In [ ]:
positive1 = df[ df['label'] == 'Positive']
positive1 = df['text_data']
negative1 = df[ df['label'] == 'Negative']
negative1 = df['text_data']

def wordcloudimp(data, color = 'black'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and not word.startswith('#')
                                and word != 'RT'
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
print("Extracted Positive words")
wordcloudimp(positive1,'white')
print("Extracted Negative words")
wordcloudimp(negative1)